# 996 小调查

In [1]:
import pandas as pd
import numpy as np
import db
import inspect
import matplotlib.pyplot as plt
import scipy.stats as stats
%matplotlib inline

In [3]:
conn=db.get_conn()
data_996 = pd.read_sql(sql="select * from _201904 where icu_996=1 and monthly_salary>0 and monthly_salary<300000 order by company_title", con=conn)
data_non_996 = pd.read_sql(sql="select * from _201904 where non_996=1 and monthly_salary>0 and monthly_salary<300000 order by company_title", con=conn)
sql_count="select COUNT(1)  from _201904 where monthly_salary>0 and monthly_salary<300000"
count=conn.execute(sql_count).fetchall()[0][0]
conn.close()

# 996有多严重

In [4]:
print('总共有效程序员招聘数据{0}条。'.format(data_996.shape[0]))
print('其中996公司共{0}家，占{1}。'.format(data_996.shape[0], data_996.shape[0]/count))
print('非996公司共{0}家，占{1}。'.format(data_non_996.shape[0], data_non_996.shape[0]/count))
print('无法确定是否996的公司占所有公司的比例为{0:.2%}。'.format(1-(data_996.shape[0]+data_non_996.shape[0])/count))

总共有效程序员招聘数据15675条。
其中996公司共15675家，占0.11221436343852013。
非996公司共30521家，占0.21849407250443847。
无法确定是否996的公司占所有公司的比例为66.93%。


因为有66%的公司情况未知，所以单单根据数据无法判断996现象有多严重。

## 名单

In [15]:
blacklist=data_996.company_title.unique()
with open('stats/blacklist.md',mode='w', encoding='utf-8') as f:
    f.write("# 996公司黑名单：\n\n\n")
    for company in blacklist:
        f.write(company+"\n\n")
    f.flush()
    f.close()
whitelist=data_non_996.company_title.unique()
with open('docs/index.html',mode='w', encoding='utf-8') as f:
    f.write("<pre>\n")
    f.write("955公司白名单：\n\n\n")
    for company in whitelist:
        f.write(company+"\n")
    f.write("</pre>\n")
    f.flush()
    f.close()
    
with open('stats/whitelist.md',mode='w', encoding='utf-8') as f:

    f.write("# 955公司白名单：\n\n\n")
    f.write("白名单生成源代码：\n\nhttps://github.com/juwikuang/china_job_survey\n\n\n")
    f.write("一下公司在招聘网站是自称是双休，或者朝九晚五，所以上了白名单。如果有不实信息的，请提高证据，我会删除名单中的这家公司。\n\n")
    f.write("本名单不关心公司是否有违法劳动法内容。至少不是996公司，都值得表扬。\n\n")
    for company in whitelist:
        f.write(company+"\n\n")

    f.flush()
    f.close()

## 工资

In [8]:
mean_996=data_996.monthly_salary.mean()
print('996公司的平均工资为{0}'.format(mean_996))
mean_non_996=data_non_996.monthly_salary.mean()
print('非996公司的平均工资为{0}'.format(mean_non_996))
compare_percentage=(mean_996-mean_non_996)*100/mean_non_996
print('996公司的工资比非996公司高{:.1f}%'.format(compare_percentage))

996公司的平均工资为15247.262991087555
非996公司的平均工资为12180.051663499118
996公司的工资比非996公司高25.2%


In [9]:
#oneway anova test
stats.f_oneway(data_996.monthly_salary, data_non_996.monthly_salary)

F_onewayResult(statistic=1294.892226831035, pvalue=1.0930844386486938e-279)

统计显示，996公司的平均工资【显著地】比非996公司高。

毕竟，现在是市场经济，人员都是自由流动的。又要别人996，又不多给钱，可能么？

是不是要996，是不是要多拿26%的工资，请君自便。

## 什么样的公司是996的？

In [10]:
del data_996['publish_date']
del data_996['published_on_weekend']
del data_996['title']
del data_996['company_title']
del data_996['company_description']
del data_996['job_description']
del data_996['job_id']

In [11]:
del data_non_996['publish_date']
del data_non_996['published_on_weekend']
del data_non_996['title']
del data_non_996['company_title']
del data_non_996['company_description']
del data_non_996['job_description']
del data_non_996['job_id']

In [12]:
data_996_sum=data_996.sum()
data_non_996_sum=data_non_996.sum()

In [13]:
def get_subset(data_sum, prefix):
    features = [feature for feature in data_996.columns if feature.startswith(prefix)]
    sub_set = data_sum[features]
    return sub_set

def get_subset_percentage(data_sum, prefix):
    sub_set=get_subset(data_sum, prefix)
    count_all=sub_set.sum()
    sub_set_percentage=sub_set/count_all
    return sub_set_percentage

def get_compare(data_996_sum, data_non_996_sum, prefix):
    data_996_percentage = get_subset_percentage(data_996_sum, prefix)
    data_non_996_percentage = get_subset_percentage(data_non_996_sum, prefix)
    data_compare=pd.DataFrame()
    data_compare['996']=data_996_percentage
    data_compare['non_996']=data_non_996_percentage
    #乘以100，会看得更清楚
    data_compare = round(data_compare * 100,1)
    return data_compare

### 公司性质

In [14]:
get_compare(data_996_sum, data_non_996_sum, 'company_type_')

,996,non_996
company_type_foreign,3.1,4.0
company_type_foreign_gov,0.0,0.0
company_type_foreign_rep,0.0,0.0
company_type_listed,9.8,8.6
company_type_non_profit,0.1,0.1
company_type_private,70.1,77.5
company_type_public_institution,0.6,0.4
company_type_startup,2.5,2.0
company_type_state,6.6,4.4
company_type_us_eu,7.2,2.9


公司性质看不出显著差异。其实，真正的外企都是双休的，但是，很多外企其实是中国人开的。统计的时候只能统计为外企。

### 公司规模

In [15]:
get_compare(data_996_sum, data_non_996_sum, 'company_size_')

,996,non_996
company_size_10000,15.0,2.0
company_size_1000_5000,13.0,9.2
company_size_150_500,17.4,25.3
company_size_50,19.3,18.2
company_size_5000_10000,2.3,1.5
company_size_500_1000,7.5,8.6
company_size_50_150,25.5,35.3


数据显示，10000人以上的大公司，更倾向于996。但是，这个可能是统计方法照成的。从996.icu收集到的996公司，很多都是大公司。

### 公司行业

In [16]:
get_compare(data_996_sum, data_non_996_sum, 'industry_')

,996,non_996
industry_ads,1.4,2.4
industry_computer,72.8,69.7
industry_edu,6.2,6.4
industry_energy,1.9,2.0
industry_finance,3.5,3.3
industry_gov,1.5,2.4
industry_logistic,1.3,1.2
industry_medical,2.4,2.9
industry_realestate,1.8,2.1
industry_service,0.9,0.8


996和非996公司并没有明显的行业分布差异。

## 城市分布

In [17]:
get_compare(data_996_sum, data_non_996_sum, 'city_')

,996,non_996
city_beijing,20.8,6.5
city_changchun,0.4,0.6
city_changsha,2.4,3.3
city_chengdu,3.9,5.5
city_chongqing,1.5,2.2
city_dalian,1.7,1.7
city_dongguan,1.0,1.0
city_fuzhou,1.2,2.1
city_guangzhou,10.2,16.2
city_hangzhou,5.9,5.7


数据显示，北京的程序员，更倾向于996。（20%对7%）

恭喜帝都上了996黑名单